<a href="https://colab.research.google.com/github/aknip/Coding-Cheatsheets/blob/main/Python-Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tl;dr

- use openpyxl: It's the only solution which can read and **update** existing Excel files. Other frameworks can only **create new** Excel files
- all Excel frameworks can not work with computed values (formulas). The initial computed cell value can be read, but it is not updated after updating the sheet. After saving the cell value will be read as "empty"!
- for full formula suppart a "real" Excel engine is needed (Windows, Mac or server solution) and a corresponing framework


In [ ]:
!pip install pandas openpyxl itables

In [ ]:
import json
import os
import textwrap
from itables import init_notebook_mode, show

init_notebook_mode(all_interactive=False)
# Display only one decimal place
pd.set_option('display.float_format', '{:.2f}'.format)
# Suppress warning messages in output
from warnings import filterwarnings
filterwarnings('ignore')

# 1. Create Excel file

In [19]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import datetime

# see https://openpyxl.readthedocs.io/en/2.4/pandas.html

# Create a Pandas dataframe with demo data
df = pd.DataFrame(columns=['Date', 'Time[d]', 'Person', 'Project Name', 'Category', 'Task', 'Jira-Tckt', 'Summary'])
df.loc[len(df)] = {'Person': 'Peter Miller', 'Date': datetime.date(2024, 1, 3) , 'Time[d]': 0.25 , 'Project Name': 'Corporate Website', 'Category': 'Design', 'Task': 'Create Template', 'Jira-Tckt': 'JIRA-1234', 'Summary': 'Worked on template' }
df.loc[len(df)] = [datetime.date(2024, 1, 4), 0.1, 'Peter Miller', 'Corporate Website', 'Design', 'Create Template', 'JIRA-1234', 'Bugfixing']
df.loc[len(df)] = [datetime.date(2024, 1, 10), 0.15, 'Peter Miller', 'Corporate Website', 'Dev', 'Create Template', 'JIRA-9876', 'Mobile navigation']
df.loc[len(df)] = [datetime.date(2024, 1, 15), 0.3, 'Peter Miller', 'Corporate Website', 'Dev', 'Create Template', 'JIRA-9877', 'Desktop navigation']
df.loc[len(df)] = [datetime.date(2024, 1, 2), 0.2, 'Maria Meyer', 'Corporate Website', 'Content', 'Homepage Content', 'JIRA-8765', 'Homepage news']
df.loc[len(df)] = [datetime.date(2024, 1, 3), 0.5, 'Maria Meyer', 'Corporate Website', 'Content', 'Homepage Content', 'JIRA-8765', 'Homepage news']
df.loc[len(df)] = [datetime.date(2024, 1, 9), 0.2, 'Maria Meyer', 'Newsletter January', 'Content', 'Corporate News', 'JIRA-5551', 'News list for newsletter']
df.loc[len(df)] = [datetime.date(2024, 1, 12), 0.6, 'Maria Meyer', 'Newsletter January', 'Design', 'Corporate News', 'JIRA-5552', 'Photos for newsletter']

# cleanup, sort
df = df.fillna('')
df = df.sort_values(['Date', 'Person'], ascending=[True, True])

# save to Excel
df.to_excel('test.xlsx', sheet_name='Test Data',index=False, header=True, engine='openpyxl')

# 2. Open Excel File and modify it: Columns widths, formating

In [21]:
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook

wb = openpyxl.load_workbook(filename = 'test.xlsx') # data_only=True
ws = wb["Test Data"] # wb.active

# Column widths
# Change width of column 2 to 4
for i in range(2,5):
    ws.column_dimensions[openpyxl.utils.get_column_letter(i)].width = 20
# Change width of single columns
ws.column_dimensions['H'].width = 30

# Column format: date, number
ws.column_dimensions['A'].number_format = 'DD.MM.YYYY' # does not work???
all_columns_cells = ws['A1:A{}'.format(ws.max_row)]
for row in all_columns_cells:
  for cell in row:
    cell.number_format = 'DD.MM.YYYY'
# or via helper function:
def style_column(worksheet, range_string, style_string):
    range_string2 = range_string.format(ws.max_row)
    all_cells = worksheet[range_string2]
    for row in all_cells:
        for cell in row:
            cell.number_format = style_string
style_column(ws, 'B2:B{}', '#,##0.00')

# Change single cells
ws['A3'].number_format = 'DD.MM.YYYY'

# Create new workshett
ws_new = wb.create_sheet('Log Data')

wb.save('test.xlsx')

# 3.1 Open Excel file as dataframe

In [26]:
df = pd.read_excel('test.xlsx', sheet_name='Test Data') # , skiprows=1, skipfooter=1 # parse_dates=['date'] # dtype={'column_name': float}
df = df.fillna('')
df['Date'] = pd.to_datetime(df['Date'], format='mixed', errors='coerce')
#df = df.drop(columns=['not-needed-1', 'not-needed-2']) # delete columns not needed
df.head()

,Date,Time[d],Person,Project Name,Category,Task,Jira-Tckt,Summary
0,2024-01-02,0.20,Maria Meyer,Corporate Website,Content,Homepage Content,JIRA-8765,Homepage news
1,2024-01-03,0.50,Maria Meyer,Corporate Website,Content,Homepage Content,JIRA-8765,Homepage news
2,2024-01-03,0.25,Peter Miller,Corporate Website,Design,Create Template,JIRA-1234,Worked on template
3,2024-01-04,0.10,Peter Miller,Corporate Website,Design,Create Template,JIRA-1234,Bugfixing
4,2024-01-09,0.20,Maria Meyer,Newsletter January,Content,Corporate News,JIRA-5551,News list for newsletter


In [28]:
print("\nIterating 1:")
for i in df.index:
    print(i, df['Person'][i],df['Summary'][i])

print("\nIterating 2:")
for index, row in df.iterrows():
    name = row['Person']
    prompt = row['Summary']
    print(f"{index}: {name}, {prompt}")


Iterating 1:
0 Maria Meyer Homepage news
1 Maria Meyer Homepage news
2 Peter Miller Worked on template
3 Peter Miller Bugfixing
4 Maria Meyer News list for newsletter
5 Peter Miller Mobile navigation
6 Maria Meyer Photos for newsletter
7 Peter Miller Desktop navigation

Iterating 2:
0: Maria Meyer, Homepage news
1: Maria Meyer, Homepage news
2: Peter Miller, Worked on template
3: Peter Miller, Bugfixing
4: Maria Meyer, News list for newsletter
5: Peter Miller, Mobile navigation
6: Maria Meyer, Photos for newsletter
7: Peter Miller, Desktop navigation


# 3.2 Updata data in dataframe and update existing Excel file

In [29]:
# Update data at index position 1 and 2
df.at[1, 'Person'] = 'Name-Updated'
df.at[2, 'Summary'] = 'Summary-Updated'
print(df.head())

        Date  Time[d]        Person        Project Name Category  \
0 2024-01-02     0.20   Maria Meyer   Corporate Website  Content   
1 2024-01-03     0.50  Name-Updated   Corporate Website  Content   
2 2024-01-03     0.25  Peter Miller   Corporate Website   Design   
3 2024-01-04     0.10  Peter Miller   Corporate Website   Design   
4 2024-01-09     0.20   Maria Meyer  Newsletter January  Content   

               Task  Jira-Tckt                   Summary  
0  Homepage Content  JIRA-8765             Homepage news  
1  Homepage Content  JIRA-8765             Homepage news  
2   Create Template  JIRA-1234           Summary-Updated  
3   Create Template  JIRA-1234                 Bugfixing  
4    Corporate News  JIRA-5551  News list for newsletter  


In [30]:

wb = openpyxl.load_workbook(filename = 'test.xlsx') # data_only=True
ws = wb["Test Data"] # wb.active

print("Updating Excel file:")
for index, row in df.iterrows():
    date = row['Date']
    time = row['Time[d]']
    person = row['Person']
    project = row['Project Name']
    category = row['Category']
    task = row['Task']
    jira = row['Jira-Tckt']
    summary = row['Summary']
    print(f"{index}: {date}, {person}")
    d = ws.cell(row=index+2, column=2, value=date)
    d = ws.cell(row=index+2, column=3, value=time)
    d = ws.cell(row=index+2, column=4, value=person)
    d = ws.cell(row=index+2, column=5, value=project)
    d = ws.cell(row=index+2, column=6, value=category)
    d = ws.cell(row=index+2, column=7, value=task)
    d = ws.cell(row=index+2, column=8, value=jira)
    d = ws.cell(row=index+2, column=9, value=summary)

# Alternative: delete data rows and append dataframe to worksheet
# continuously delete row 2 until there is only first row (header row) is left over
# while(ws.max_row > 1):
#     ws.delete_rows(2) # removes the row 2
# append dataframe to worksheet
# for row in dataframe_to_rows(df, index=False, header=False):
#     if row != [None]:
#        ws.append(row)

# Save the Excel file.
wb.save('test.xlsx')

Updating Excel file:
0: 2024-01-02 00:00:00, Maria Meyer
1: 2024-01-03 00:00:00, Name-Updated
2: 2024-01-03 00:00:00, Peter Miller
3: 2024-01-04 00:00:00, Peter Miller
4: 2024-01-09 00:00:00, Maria Meyer
5: 2024-01-10 00:00:00, Peter Miller
6: 2024-01-12 00:00:00, Maria Meyer
7: 2024-01-15 00:00:00, Peter Miller


# 3.2.1 Updata data in Dataframe and write Excel file / Notes

Pandas to existing Excel

How to write data to an arbitrary location in an existing Excel workbook

1. manual, iterating:

https://stackoverflow.com/questions/72669750/how-to-update-an-existing-excel-spreadsheet-with-data-from-dataframe

2. df.to_excel

https://stackoverflow.com/questions/69628517/put-pandas-data-frame-to-existing-excel-sheet

3. General code examples: How to Process Excel Data in Python and Pandas

1 How to Process Excel Files with Python?
2 How to Create an Excel File in Python?
3 Export Pandas Dataframe to Excel Sheet in Python
4 Create an Excel File With Multiple Sheets in Python
5 How to Read Excel Data in Python?
6 Update an Excel File in Python
7 Performing Delete Operations on an Excel File
8 Merge Excel Sheets Into a CSV File in Python
9 Conclusion

https://python.land/data-processing/process-excel-data-in-python-and-pandas


# 2. Read Excel file (without  dataframe)

In [37]:
workbook = load_workbook(filename = 'test.xlsx') # data_only=True
worksheet = workbook['Test Data'] # workbook.active

# iterate through headers in row 1
# create dictionary to access columns
column = {}
for column_index, cell in enumerate(worksheet["1"]):
  column[cell.value] = column_index + 1

# iterate through all lines and rows
for row_index, row in enumerate(worksheet):
  for column_index, cell in enumerate(row):
    cell = worksheet.cell(row=row_index+1, column=column_index+1)
    if cell.value != None:
      print(cell.value)

# Get column nr for header name
print(column["Person"])

# read data
cell = worksheet.cell(row=2, column=2)
cell = worksheet["B2"]
print(cell.value)

# write data
cell.value = "Test"

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')

Date
Time[d]
Person
Project Name
Category
Task
Jira-Tckt
Summary
2024-01-02 00:00:00
2024-01-02 00:00:00
0.2
Maria Meyer
Corporate Website
Content
Homepage Content
JIRA-8765
Homepage news
2024-01-03 00:00:00
2024-01-03 00:00:00
0.5
Name-Updated
Corporate Website
Content
Homepage Content
JIRA-8765
Homepage news
2024-01-03 00:00:00
2024-01-03 00:00:00
0.25
Peter Miller
Corporate Website
Design
Create Template
JIRA-1234
Summary-Updated
2024-01-04 00:00:00
2024-01-04 00:00:00
0.1
Peter Miller
Corporate Website
Design
Create Template
JIRA-1234
Bugfixing
2024-01-09 00:00:00
2024-01-09 00:00:00
0.2
Maria Meyer
Newsletter January
Content
Corporate News
JIRA-5551
News list for newsletter
2024-01-10 00:00:00
2024-01-10 00:00:00
0.15
Peter Miller
Corporate Website
Dev
Create Template
JIRA-9876
Mobile navigation
2024-01-12 00:00:00
2024-01-12 00:00:00
0.6
Maria Meyer
Newsletter January
Design
Corporate News
JIRA-5552
Photos for newsletter
2024-01-15 00:00:00
2024-01-15 00:00:00
0.3
Peter Miller
Co